### <p style="font-family:Garamond,serif">Following the steps in NLP course notebook week 2.2, I first conduct LDA to the tweets I scrapped from #AstraZeneca and #Pfizer, then continue on generation visuialtion with pyLDAvis.</p>

In [ ]:
!pip install pyldavis

In [1]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud as wc
import pandas as pd 
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction import _stop_words
from sklearn.metrics.pairwise import cosine_similarity as cosine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.tokenize.casual import casual_tokenize
import pandas as pd
from nlpia.data.loaders import get_data
from sklearn.decomposition import TruncatedSVD

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/pugnlp/constants.py:136: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  [datetime.datetime, pd.datetime, pd.Timestamp])
/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/pugnlp/constants.py:158: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  MIN_TIMESTAMP = pd.Timestamp(pd.datetime(1677, 9, 22, 0, 12, 44), tz='utc')
/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/pugnlp/tutil.py:100: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  np = pd.np
/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/pugnlp/util.py:80: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Imp

In [2]:
import re 
import nltk 
import numpy as np

In [3]:
from __future__ import print_function

In [4]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [34]:
#column_drop = data_from_csv.loc[:,'Unnamed: 0']
#column_drop

0            0
1            1
2            2
3            3
4            4
         ...  
23863    23863
23864    23864
23865    23865
23866    23866
23867    23867
Name: Unnamed: 0, Length: 23868, dtype: int64

In [10]:
file_path = '../data/en_hashtag_astrazeneca.csv'
data_from_csv = pd.read_csv(file_path, encoding='utf-8') #Reads a CSV file into a pandas dataframe

data_from_csv

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


FileNotFoundError: [Errno 2] No such file or directory: '../data/en_hashtag_astrazeneca.csv'

In [23]:
np.shape(data_from_csv) 

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(23868, 1)

In [24]:
df = data_from_csv.rename(columns={'text': 'text'})
df

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text
0,Those who got their vaccines like me in the fi...
1,"#Pfizer , Johnson&amp;Johnson , #Moderna , #As..."
2,Crytoverse and Metaverse face any risk on shor...
3,What happened to the Astra Zeneca vaccine? #As...
4,"I am not anti vaccine or,as they insult a #COV..."
...,...
23863,This is a move that will come to bite these co...
23864,🇩🇪#Germany &amp; 🇨🇵#France joining 13 other co...
23865,#AstraZeneca #Bundesgesundheitsminister please...
23866,I would also like to hear from anyone that has...


In [25]:
#Louis' tokenizer 
lem = WordNetLemmatizer()


def my_tokeniser(doc):
    #Split on spaces
    tokens = re.split(r'[-\s.,;!?]+', doc)
    processed = []
    for t in tokens:
        t = lem.lemmatize(t.lower()) 
        if not t in _stop_words.ENGLISH_STOP_WORDS:
            processed = processed + [t]
    #Return an array of tokens for that document
    return processed

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
#Create count vectors
cv = CountVectorizer(min_df=5, tokenizer=my_tokeniser,)

count_array = cv.fit_transform(df['text'].values.astype('U'))
count_array

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<23868x7012 sparse matrix of type '<class 'numpy.int64'>'
	with 330605 stored elements in Compressed Sparse Row format>

In [27]:
#create tfidf vectors from count vectors
transformer = TfidfTransformer()
transformer.fit(count_array)
tfidf = transformer.transform(count_array)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
tfidf_vectorizer = TfidfVectorizer(**cv.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(df['text'].values.astype('U'))
print(dtm_tfidf.shape)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:1805: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  warnings.warn("Only {} 'dtype' should be used. {} 'dtype' will "


(23868, 7012)


In [29]:
tfidf_docs = tfidf.toarray()
tfidf_docs

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.07459704, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [30]:
len(cv.vocabulary_)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


7012

In [31]:
cv.vocabulary_

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'got': 3573,
 'vaccine': 6524,
 'like': 4301,
 'round': 5567,
 'card': 2345,
 'qr': 5252,
 'code': 2509,
 'dose': 3015,
 '2': 965,
 "doesn't": 2992,
 'hear': 3708,
 'hospital': 3795,
 '#covidvaccine': 243,
 '#astrazeneca': 68,
 '#pfizer': 552,
 'johnson&amp': 4107,
 'johnson': 4106,
 '#moderna': 476,
 'face': 3303,
 'risk': 5542,
 'shortage': 5778,
 'vax': 6545,
 'happened': 3661,
 'astra': 1982,
 'zeneca': 6835,
 '#covid19': 195,
 '#booster': 126,
 'anti': 1877,
 'having': 3688,
 'jab': 4072,
 'flu': 3416,
 'protect': 5196,
 'pneumonia': 5026,
 'begin': 2104,
 'question': 5263,
 'booster': 2200,
 'body': 2186,
 '\\nno': 1639,
 'hate': 3681,
 '#covid': 194,
 'just': 4134,
 'boosted': 2199,
 '#pzifer': 581,
 "didn't": 2915,
 '#vaccinated': 715,
 'impressive': 3905,
 '@nhsuk': 1302,
 'text': 6231,
 'message': 4495,
 'accelerate': 1704,
 '#vaccine': 727,
 '18th': 952,
 'december': 2820,
 '4th': 1066,
 'wondering': 6754,
 '💉💉': 6908,
 '#getvaccinatednow': 339,
 '#omicron': 530,
 'it’s': 4

In [32]:
# Get the column_numbers for each term in our vocabulary
column_nums, terms = zip(*sorted(zip(cv.vocabulary_.values(), cv.vocabulary_.keys())))
tfidf_docs_df = pd.DataFrame(tfidf_docs, columns=terms, index=df.index)
tfidf_docs_df #show it

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,,"""","""#astrazeneca","""\n#astrazeneca","""a","""alternative""","""astrazeneca","""blood","""fully","""i",...,🤷🏽\u200d♂️,🥰,🥳,🥴,🥵,🥶,🥺,🦠,🧐,🧵
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23863,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23864,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23865,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23866,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
#Centres vectorized documents by subtracting the mean
tfidf_docs_df_mean_subtracted = tfidf_docs_df - tfidf_docs_df.mean()

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
svd = TruncatedSVD(n_components =20 , n_iter = 100) 
svd_topic_vectors = svd.fit_transform(tfidf_docs_df.values)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
svd
#svd_topic_vectors

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TruncatedSVD(n_components=20, n_iter=100)

In [36]:
#put it in a dataframe, again being good to ourselves by giving it row and column names
svd_topic_vectors_df = pd.DataFrame(svd_topic_vectors, index=df.index, columns=['topic{}'.format(i) for i in range(20)])
svd_topic_vectors_df

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15,topic16,topic17,topic18,topic19
0,0.127126,0.066884,-0.033984,0.046460,-0.030515,0.032919,-0.027415,-0.050427,-0.001146,-0.001147,0.032888,0.044516,-0.020498,-0.005532,-0.031863,-0.013524,0.009732,0.041408,-0.040319,0.008417
1,0.087712,-0.026008,0.028120,0.023027,-0.084003,-0.077882,-0.053446,0.133705,-0.021749,-0.084466,0.036749,0.173297,0.065106,0.028239,-0.043975,-0.180965,-0.070736,-0.055340,0.020514,-0.062124
2,0.074724,-0.054940,0.008498,0.010629,0.044371,0.023052,0.018223,0.012531,-0.008576,-0.013840,0.005486,0.012292,0.015366,-0.006712,0.020169,0.013427,0.021133,0.015014,0.041112,0.039513
3,0.148647,-0.070302,0.002963,0.021329,-0.093072,-0.037401,-0.079090,-0.005603,-0.018174,0.027142,0.012663,0.037481,0.018544,-0.097605,-0.057595,0.060523,0.026228,-0.003734,0.001228,0.009191
4,0.170570,-0.028080,-0.015887,-0.131185,0.047519,-0.127484,-0.128368,0.063593,0.037712,-0.029137,-0.022516,0.049730,-0.000954,0.017458,-0.112220,-0.043188,-0.049171,0.033507,-0.046457,-0.014496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23863,0.082584,-0.051649,0.006585,0.006564,-0.016680,-0.019879,0.025495,0.003084,0.012326,-0.025180,-0.003881,-0.009220,-0.043364,-0.007450,0.013954,0.068710,0.024321,-0.024104,-0.020548,0.009891
23864,0.068859,-0.043146,0.009828,-0.001561,-0.021563,-0.027214,-0.016815,0.009311,-0.002921,-0.017396,0.000735,0.006980,-0.041169,-0.007088,0.022905,0.035824,0.031123,-0.069816,0.002897,-0.005043
23865,0.065859,-0.056135,0.012505,0.011899,0.006562,-0.000111,0.008972,0.014301,0.011781,-0.017596,0.006263,0.014798,-0.041526,-0.003206,0.033335,-0.014337,0.122342,0.050247,0.005764,-0.026375
23866,0.146380,-0.071289,-0.018957,0.025721,-0.065287,-0.045437,-0.032904,-0.040794,-0.010972,0.050952,-0.011134,-0.039530,-0.022938,-0.055684,-0.029435,0.087262,0.006056,-0.053180,-0.095638,-0.017131


In [37]:
#below, we use .T to transpose svd.components_ into the familiar form where *rows* correspond to words and *columns* to topics
topic_weights = pd.DataFrame(svd.components_.T, index=terms, columns=['topic{}'.format(i) for i in range(20)])
topic_weights #display it

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15,topic16,topic17,topic18,topic19
,0.238144,-0.096366,-0.062814,-0.069860,-0.008621,-0.004755,0.627836,-0.067331,0.282193,-0.238321,-0.202251,-0.027248,0.203063,0.301717,-0.120035,-0.088639,-0.083228,-0.099104,-0.082928,-0.045113
"""",0.010018,-0.013177,0.000884,0.004997,-0.000223,-0.000117,-0.004252,0.000397,-0.000143,0.001326,0.001581,-0.001100,-0.007446,-0.001832,0.004417,0.011317,0.004248,-0.003796,0.002664,0.004200
"""#astrazeneca",0.000566,-0.001031,-0.000213,0.000087,-0.000644,-0.000408,0.000041,-0.000273,0.000844,0.000087,-0.001196,-0.001396,-0.000945,0.000497,-0.000915,-0.000257,0.000576,-0.000382,-0.000795,-0.000048
"""\n#astrazeneca",0.000320,-0.000646,-0.000119,0.000531,0.000616,0.000360,-0.000507,-0.000065,-0.000511,0.000388,0.000076,-0.000200,-0.000645,0.000151,0.000174,-0.000573,-0.000322,0.000039,0.000242,0.000258
"""a",0.000448,-0.000792,0.000352,0.000336,0.000075,-0.000087,0.000339,-0.000108,-0.000019,-0.000298,-0.000184,0.000081,-0.000672,0.000416,0.000050,0.000945,0.000134,0.000023,0.000494,-0.000055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
🥶,0.000425,0.000045,-0.000164,-0.000991,0.000148,-0.000696,0.000998,-0.001225,-0.001664,0.000482,0.000878,0.000752,0.001322,0.000283,-0.001625,0.001198,0.001272,-0.001058,0.003394,-0.000990
🥺,0.001116,-0.000022,0.000385,-0.001584,-0.000330,-0.000821,0.000283,0.000340,-0.000625,0.001906,-0.000461,-0.000554,0.001189,-0.002500,-0.001794,-0.000542,0.002137,0.002380,0.002414,-0.000037
🦠,0.001511,-0.000179,0.000084,-0.000162,-0.000498,-0.000451,-0.001937,0.000299,-0.000593,-0.000009,-0.000064,0.001775,0.001873,0.001765,0.002316,-0.000150,0.000979,0.001494,0.000438,0.002381
🧐,0.000881,-0.001159,-0.000229,-0.000441,0.001368,0.000988,-0.000012,-0.001292,-0.001003,0.000874,0.000065,0.000260,-0.000080,0.000663,-0.000547,0.000369,0.000609,0.001305,0.001023,-0.000626


In [38]:
topic_weights.topic2.sort_values(ascending=False)[:20] # show top 10 weighted words for topic 2

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


vaccinated           0.691695
fully                0.618601
#astrazeneca         0.078520
#vaccinated          0.070151
finally              0.040919
#covid19             0.040607
people               0.040022
#pfizer              0.035586
#moderna             0.034330
💉                    0.034242
vaxxed               0.032346
\n#astrazeneca       0.030760
16+                  0.027970
#getvaccinatednow    0.025428
70+                  0.024313
0+                   0.024282
#getvaccinated       0.023041
i'm                  0.019749
officially           0.018799
clot                 0.017617
Name: topic2, dtype: float64

In [39]:
cv = CountVectorizer(min_df=5, tokenizer=my_tokeniser,)
count_array = cv.fit_transform(df['text'].values.astype('U'))
count_array.shape

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(23868, 7012)

In [40]:
lda = LatentDirichletAllocation(n_components=20,
                                random_state=123,
                                learning_method='batch')
lda.fit(count_array)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(random_state=123)

In [24]:
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(count_array)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(n_components=20, random_state=0)

In [41]:
# And run LDA. This could take a loooonggggg time....
lda_topics = lda.fit_transform(count_array)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### <p style="font-family:Garamond,serif"> Generate visualitaion with pyLDAvis </p>

In [42]:
pyLDAvis.sklearn.prepare(lda, count_array, cv)

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.192745  0.097169       1        1  12.638243
3      0.215164  0.060032       2        1  12.099897
2     -0.088062  0.091569       3        1  11.356193
4     -0.088136 -0.004394       4        1  11.007817
6      0.160927  0.000722       5        1  10.379078
8     -0.167662 -0.094298       6        1   9.435148
1      0.074742  0.024723       7        1   9.376722
7     -0.038152  0.067138       8        1   8.981471
5      0.106246  0.028966       9        1   8.576798
0      0.017678 -0.271626      10        1   6.148633, topic_info=                Term         Freq        Total Category  logprob  loglift
6519      vaccinated  1679.000000  1679.000000  Default  30.0000  30.0000
2493            clot  1878.000000  1878.000000  Default  29.0000  29.0000
2174           blood  1878.000000  1878.000000  Default  28.0000  28.0000
552          #pfizer  2028.000000  2028.000000  Default  27.0000  27.0000
3499           fully   699.000000   699.000000  Default  26.0000  26.0000
...              ...          ...          ...      ...      ...      ...
6541         variant    72.235711   227.871997  Topic10  -5.6880   1.6401
964              1st    93.907073   911.715101  Topic10  -5.4256   0.5159
4134            just    83.994468  2350.333168  Topic10  -5.5372  -0.5426
1363  \n#astrazeneca    77.058487   740.037334  Topic10  -5.6233   0.5268
194           #covid    75.948205   657.775800  Topic10  -5.6379   0.6301

[762 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.179907     
0         2  0.147944     
0         3  0.120364     
0         4  0.154337     
0         5  0.111780     
...     ...       ...  ...
6968      6  0.017642    🙏
6968      9  0.864444    🙏
6968     10  0.070567    🙏
7004      2  0.075774    🥳
7004     10  0.909288    🥳

[1880 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 4, 3, 5, 7, 9, 2, 8, 6, 1])

### <p style="font-family:Garamond,serif"> Save the file into html for better viewing, please check the html file for both visualization. :)</p>

In [44]:
vis_en_hashtag_astrazeneca = pyLDAvis.sklearn.prepare(lda, count_array, cv)
pyLDAvis.save_html(vis_en_hashtag_astrazeneca , "vis_en_hashtag_astrazeneca.html")

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
